In [ ]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver
from langchain.tools import tool

# 別途必要なモジュール
import os
import re
from pathlib import Path
from langchain_core.messages import ToolMessage, AIMessageChunk


# ===== Stateクラスの定義 =====
# langchain では LLM に型を伝えるために typing で Annotation するのがミソ
class State(TypedDict):
    messages: Annotated[list, add_messages]


# ===== グラフの構築 =====
def build_graph(model_name: str):
    # 実質的には ChatOpenAI(model_name=...) と同等です
    llm = LangchainAIClientGenerator.generateChatOpenAI(model_name=model_name)

    # 外部ツールを定義して追加
    tools = []
    tools.append(UserDefTool.get_tool())
    
    # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    # すみません、Tavily のアカウント取得ができなかったため
    # 実際に以下の Tavily 外部ツールは試せていません
    # 以下のようにすれば動くはずですが、もしも動作しなければご報告をいただけますでしょうか?
    # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    from langchain_community.tools.tavily_search import TavilySearchResults
    tools.append(TavilySearchResults(max_results=2))
    # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

    # 外部ツールを LLM にバインド
    llm_with_tools = llm.bind_tools(tools)
    tool_node = ToolNode(tools)

    # グラフの作成
    graph_builder = StateGraph(State)

    # グラフのノードを作成
    node = ChatBotNode(llm_with_tools)
    graph_builder.add_node(node.get_name(), node.get_node()) # チャットボットの追加
    graph_builder.add_node("tools", tool_node)               # ツールの追加

    # ------------------
    # ノードの結線
    # ------------------
    # 条件付エッジの作成
    # チャットボット --(必要なら)--> ツールへ進むステップを追加
    graph_builder.add_conditional_edges(
        "chatbot",
        tools_condition, # ツール呼出と判断したらツールノードたち (=tools) を呼ぶ
    )

    # ツール ---> チャットボットへ戻るステップを追加
    graph_builder.add_edge("tools", "chatbot")

    # 開始ノードの指定
    graph_builder.set_entry_point(node.get_name())

    # 終了ノードの指定 (ループするので不要?)
    # graph_builder.set_finish_point(node.get_name())

    # ------------------
    # グラフのビルド
    # ------------------
    # やり取りを記憶してもらう
    memory = MemorySaver()

    # 実行可能なステートグラフの作成
    graph = graph_builder.compile(checkpointer=memory)
    
    # 可視化もできる
    # from IPython.display import Image, display
    # display(Image(graph.get_graph().draw_mermaid_png()))

    # グラフを返す
    return graph


# ===== グラフ実行関数 =====
def stream_graph_updates(graph: StateGraph, user_input: str):
    # Prompt (state) をつくる
    messages = {
        "messages": [
            ("system", "回答は名古屋弁でお願いします"),
            ("user", user_input)
        ]
    }

    # Memo. もしも streaming しなくてよいならば以下で表示可能
    # response = graph.invoke(
    #     messages,
    #     {"configurable": {"thread_id": "1"}}
    # )
    # for message in response["messages"]:
    #     print(message)
    #     print("==================")


    # やりとりを記憶しておくために configurable を指定し stream_mode を設定する
    # values   ... すべてのメッセージの履歴をストアしてくれる
    # updates  ... ストアするのは前回取得との差分のみ
    # messages ... LLM の回答のみ
    events = graph.stream(
        messages,
        {"configurable": {"thread_id": "1"}},
        stream_mode="messages"
    )

    # 結果をストリーミングで得る (+表示)
    print("\033[32m" + f"質問: {user_input}" + "\033[0m")
    print("---------------------------")
    
    used_tools = []
    llm_color  = "\033[34m"

    for event in events:        
        chunk = event[0]

        # ツールを使ったときの回答 (非同期だとどうなるんでしょう?)
        if isinstance(chunk, ToolMessage):
            print("\033[33m" + f"外部ツール {chunk.name} が呼び出されました ::: {chunk.content}" + "\033[0m")
            if chunk.name not in used_tools:
                used_tools.append(chunk.name)
                llm_color  = "\033[35m"
    
        # LLM からの回答をリアルタイムで表示する
        elif isinstance(chunk, AIMessageChunk):
            if llm_color:
                # 外部ツールを使っているならはじめに報告する
                if len(used_tools) > 0:
                    print("\033[1;35m" + f"◆◆◆ 以下の回答は外部ツール {", ".join(used_tools)} から得られた答えを使っています ◆◆◆" + "\033[0m")
                # LLM の回答の色をセット
                print(llm_color, end="", flush=True)
                llm_color = None
    
            print(chunk.content, end="", flush=True)

    # 色のリセット・改行
    print("\n".join([ "\033[0m", "==========================="]))



# ===== メイン実行ロジック =====
def main():
    # モデル名
    MODEL_NAME = "gpt-4o-mini" 

    # 環境変数の読み込み
    load_dotenv("../.env")
    os.environ['OPENAI_API_KEY'] = os.environ['API_KEY']

    # グラフの構築
    graph = build_graph(MODEL_NAME)

    # メインループ
    while (user_input := input("質問:")) != "":
        stream_graph_updates(graph, user_input)

    print("ありがとうございました!")


### =======================================
### 以下、ユーティリティクラスとなります
### =======================================

# ChatOpenAI のインスタンスを入手するためのクラス
class LangchainAIClientGenerator:

    OPEN_API_ENV_NAME = "API_KEY"
    
    # この関数を呼び出すことで .env ファイルもしくは環境変数から OpenAI API キーをロードして
    # ChatOpenAI の client インスタンスを返します
    # api_key_validation      ... True なら簡易的なキーのチェックを行います
    # display_debug_messages  ... True ならデバッグメッセージを表示します
    # **kwargs                ... (model_name) モデルの指定などの langchain_openai.ChatOpenAI に渡すパラメータ
    @classmethod
    def generateChatOpenAI(cls, **kwargs):
        kwargs["api_key"] = cls.__load_api_key()
        return ChatOpenAI(**kwargs)
    

    # この関数で OpenAI API のキーを読み取ります
    # 1. 実行パスの ../.env が存在するならば API_KEY を環境変数としてロードして読み取る
    # 2. 環境変数の API_KEY をロードして読み取る
    # 3. API_KEY の中身の値が絶対ファイルパスならばその中身をそのままロードする
    #    そうでないならば中身をそのまま API キーとして使う
    # 4. キーの簡易的なフォーマットチェック
    @classmethod
    def __load_api_key(cls):        
        api_key = None

        # 1. ../.env を読み取る
        env_file_path = Path().resolve().parent.resolve() / ".env"
        if env_file_path.is_file():
            try:
                load_dotenv(env_file_path)
            except:
                raise Exception("Found .env file but failed to load dotenv file! Please install python-dotenv module.")

        
        # 2. API_KEY を読み取る
        api_key = os.environ.get(cls.OPEN_API_ENV_NAME, None)

        # 3. API_KEY の中身チェック
        api_file_path = Path(api_key).expanduser()
        if (api_file_path.is_absolute() and api_file_path.is_file()):
            with open(api_file_path, "r") as f:
                api_key = f.read().strip()

        # 4. キーの簡易チェック
        if re.match(r"^sk\-.*$", api_key) is None:
            raise Exception("Failed to load api key!")
        
        return api_key


# ChatBot 用のクラス、ノードの役割を果たします
class ChatBotNode:
    def __init__(self, llm):
        self.llm = llm
        self.name = "chatbot"


    def chatbot(self, state):
        return {
            "messages": [
                self.llm.invoke(state["messages"])
            ]
        }

    def get_node(self):
        return self.chatbot


    def get_name(self):
        return self.name


# もしも外部ツールを自作したい場合について以下のように @tool アノテーションを使って関数を引き渡す??
# Memo. もしも自作のツールを作って連動したい場合は以下のようにツールを作ることができる
class UserDefTool:
    # Langchain ではアノテーション (****:int) 部分で LLM に型を渡すので必須
    # ※ そうでなくても型の明示的な宣言は良い習慣
    @tool
    @staticmethod
    def get_now(is_utc : bool = False) -> str:
        """現在時刻を返します
        Args:
            is_utc: もしも協定世界時(UTC)もしくはグリニッジ標準時(GMT)を指定された場合は true としてください、そうでなければ false を代入してください
        """
        # 上のドキュメントも必要です (ただし LLM にわたるので関数のドキュメント化とは少し違います)

        from datetime import datetime, timezone
        now = datetime.now(timezone.utc) if is_utc else datetime.now()
        now_str = now.strftime("%Y-%d-%m %H:%M%:%S")
        response = f"現在時刻は {now_str} です。 引数として utc={is_utc} を指定しました。"
        return response
    
    
    @classmethod
    def get_tool(cls):
        return cls.get_now


if __name__ == "__main__":
    main()

質問: グリニッジ標準時とUTCは何が違うのでしょうか?
---------------------------
グリニッジ標準時（GMT）と協定世界時（UTC）は、実際にはほとんど同じように使われることが多いけど、ちょっとした違いがあるんだわ。

まず、GMTは、イギリスのグリニッジ天文台を基準にした時間で、主に天文学や航海のために使われてたんよ。一方、UTCは、国際的な標準時で、原子時計を基準にしてるから、より精密なんだわ。

さらに、GMTは時間帯の一つとして使われることが多いけど、UTCは時間の標準として使われるから、例えば、UTC+9とかのように、他の時間帯との比較で使われることが多いわけさ。

まとめると、GMTは歴史的な基準で、UTCは現代の標準って感じやね。
質問: なるほどです、では現在時刻とグリニッジ標準時を教えていただけますか?
---------------------------
外部ツール get_now が呼び出されました ::: 現在時刻は 2026-26-02 16:02:31 です。 引数として utc=False を指定しました。
外部ツール get_now が呼び出されました ::: 現在時刻は 2026-26-02 07:02:31 です。 引数として utc=True を指定しました。
◆◆◆ 以下の回答は外部ツール get_now から得られた答えを使っています ◆◆◆
今の時刻は、名古屋での時間が「2026年2月26日 16時02分31秒」やわ。グリニッジ標準時（GMT）は「2026年2月26日 07時02分31秒」やで。時間の差があるから、注意せんといかんね！
ありがとうございました!
